## Assignment 4 - Text classification

The assignment for this week builds on these concepts and techniques. We're going to be working with the data in the folder CDS-LANG/toxic and trying to see if we can predict whether or not a comment is a certain kind of toxic speech. You should write two scripts which do the following:

-->The first script should perform benchmark classification using standard machine learning approaches
This means CountVectorizer() or TfidfVectorizer(), LogisticRegression classifier

-Save the results from the classification report to a text file

In [2]:
#setup script wasn't working in class, so:
!pip install nltk beautifulsoup4 contractions tensorflow scikit-learn

  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)
  Using cached contractions-0.1.68-py2.py3-none-any.whl (8.1 kB)
  Using cached tensorflow-2.8.0-cp39-cp39-manylinux2010_x86_64.whl (497.6 MB)
  Using cached scikit_learn-1.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 KB 4.4 MB/s eta 0:00:00a 0:00:01
  Using cached textsearch-0.0.21-py2.py3-none-any.whl (7.5 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 27.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 KB 1.2 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 KB 299.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 30.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 KB 2.1 MB

In [1]:
#to train a Logistic Regression model using scikit-learn, from vis_assign_2

#session_8
# system tools
import os
import sys
sys.path.append(os.path.join("..", "..", "CDS-LANG"))

# data munging tools
import pandas as pd
import utils.classifier_utils as clf

# Machine learning stuff
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier 
from sklearn.model_selection import ShuffleSplit
from sklearn import metrics

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

#surpress warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" #ignore warnings
    
#scikit-learn
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

# data wranling
import numpy as np

import os

In [2]:
# Load the data:
# get the filepath for test labels
filepath = os.path.join("..","..","CDS-LANG","toxic","VideoCommentsThreatCorpus.csv")

In [3]:
#open csv with pandas
data = pd.read_csv(filepath)

In [4]:
#looking at the dataset
print(data)
#1 = toxic, 0= non-toxic

       label                                               text
0          0  It's because Europeans do not want to change t...
1          0  The Muslims there do not want to assimilate pr...
2          1  But it's ok....because Europe will soon rebel ...
3          0  I forsee a big civil war in Europe in the futu...
4          0  ISLAM – A Simple, Humanitarian and Attractive ...
...      ...                                                ...
28638      1  yeah we are all monsters..I'm gonna kill u rig...
28639      0                       stupid brainwashed idiot..\n
28640      0  have you EVER been to Serbia or kosovo...fucki...
28641      0  probably u mean to this monsters, fucker /watc...
28642      0  the fucking funniest thing is that fucking ame...

[28643 rows x 2 columns]


In [5]:
#create more balanced dataset, getting a random sample of the dataset
data_balanced = clf.balance(data, 1000) #1000 datapoints for each

In [6]:
# create new variables called text and label
# taking the data out of the dataframe so that we can mess around with them.
X = data_balanced["text"] #text column
y = data_balanced["label"] #label column

In [7]:
#Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, #texts for the model
                                                    y, #classification labels
                                                    test_size = 0.2, #create an 80/20 split (testing to be 20% of the overall data)
                                                    random_state = 42) #where we should start: just for reproducability

In [8]:
# Create vectorizer object
#can modify all of these parameters (or not set any)
vectorizer = TfidfVectorizer(ngram_range = (1,2), #the size of tokens: unigrams and bigrams (1: individual words and 2 word pairings: repeated word pattern)
                             lowercase = True, #use lowercase: getting all instances of the word
                             max_df = 0.95, #remove very common words, stop words, if these words appear in over 95% of the documents= gone
                             min_df = 0.05, #remove very rare words, occur in fewer than 0.05% of the documents= gone, does not "discriminate"
                             max_features = 100) #keep only top 500 features (compress to focus on the 500 most frequent words- further filtering)

In [9]:
#This vectorizer is then used to turn all of our documents into a vector of numbers, instead of text.
#use on all documents
#first we fit to the training data
X_train_feats = vectorizer.fit_transform(X_train)

#we then do it for our test data
X_test_feats = vectorizer.transform(X_test) #no fitting!! as we are trying to predict

#get feature names: the unigrams and bigrams
feature_names = vectorizer.get_feature_names()

In [10]:
#Classifying and predicting
classifier = LogisticRegression(random_state=42).fit(X_train_feats, y_train)

In [11]:
#use the classifier to make predictions
y_pred = classifier.predict(X_test_feats)

In [12]:
# Evaluate
classifier_metrics = metrics.classification_report(y_test, y_pred)
print(classifier_metrics)

              precision    recall  f1-score   support

           0       0.69      0.80      0.74       199
           1       0.77      0.64      0.70       201

    accuracy                           0.72       400
   macro avg       0.73      0.72      0.72       400
weighted avg       0.73      0.72      0.72       400



In [14]:
#Save the results from the classification report to a text file
g = open("../../cds-lang/Lang-assignments/output/benchmark_assign_4.txt",'w')
print(classifier_metrics, file=g)

print("Done! classifier_metrics has been generated and saved in the output folder as benchmark_assign_4.txt")

Done! classifier_metrics has been generated and saved in the output folder as benchmark_assign_4.txt
